In [34]:
from pandas import DataFrame
import numpy as np
from copy import deepcopy
import os
from sklearn.metrics import mean_absolute_error
import logging
from typing import Type

import numpy as np
import warnings
import os
import cv2
import pickle
import pandas as pd
import scipy as sp
import scipy.misc
from matplotlib import pyplot as plt
from glob import glob
import tensorly as tl
import numpy as np
from sklearn.linear_model import orthogonal_mp
from tensorly.decomposition import tucker, partial_tucker
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)

In [64]:
os.mkdir("images_perm") 

In [81]:
def convert_weather_value_to_px(left_border, value):
    if isinstance(value, float):
        value *= 100
        left_border *= 100
        left_border, value = int(left_border), int(value)
    b = 0 if value >= 0 else 255
    shifted_value = abs(value) - left_border
    r = shifted_value // 256
    g = shifted_value % 256
    return [0, 0, r, g]


def convert_snp_to_px(placed, value, bits):
    r, g, b = 0, 0, 0
    if value == 1:
        r = (1 << placed) | bits
        g = 0
    elif value == 0:
        r = 0
        g = 0
    elif value == 2:
        r = (1 << placed) | bits
        g = (1 << placed) | bits
    return [math.sqrt(r) if math.sqrt(r) <= 255 else 255, math.sqrt(g) if math.sqrt(g) <= 255 else 255], bits


def transform(df,ni):
    num_of_image = 1
    idx1 = [i for i in range(2, 20)]
    idx2 = [i for i in range(23, 148)]
    dir_ = "images_perm/images_"+str(ni)
    for f in os.listdir(dir_):
        os.remove(os.path.join(dir_, f))
    os.rmdir(dir_)
    os.mkdir(dir_)
    with open("_.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(df.shape[0]):
            pixels1 = []
            pixels2 = []
            snp_pixels = []
            placed = 0
            temp_bit = 0
            for c in columns[2:20]:
                temp_array, temp_bit = convert_snp_to_px(placed, df.iloc[i:i+1,idx1][c][0], temp_bit)
                snp_pixels.append(temp_array)
                
                placed += 1
            index_snp = 0
            repeat_5 = 1
            day_pixels = []
            for w in w_columns:
#                 
                tmp = convert_weather_value_to_px(0, df.iloc[i:i+1,idx2][w][0])
                tmp[0] = snp_pixels[index_snp][0]
                tmp[1] = snp_pixels[index_snp][1]
                day_pixels.append(tmp)
                if index_snp > 16:
                    index_snp = 0
                index_snp += 1
                if repeat_5 % 5 == 0:
                    pixels1.append(day_pixels)
                    pixels2.append(day_pixels)
                    day_pixels = []
                repeat_5 = repeat_5 + 1
            pixels = np.zeros((25, 25, 4))
            for i in range(25):
                for j in range(25):
                    pixels[i][j] = pixels1[i][j % 5]
            pixels = np.array(pixels, dtype=np.uint8)
            new_image = Image.fromarray(pixels, mode='RGBA').rotate(90, expand=True)
            # путь относительно текущего скрипта
            
            new_image.save('images_perm/images_'+str(ni) + "/image" + str(num_of_image) + ".png")
            num_of_image += 1

In [91]:
class Pursuit:
    """
    Algorithms that inherit from this class are methods to solve problems of the like
    \min_A \| DA - Y \|_2 s.t. \|A\|_0 <= t.
    Here, D is a given dictionary of size (n x K)
    Y is a given matrix of size (n x N), where N is the number of samples
    The Pursuit will return a matrix A of size (K x N).
    """

    def __init__(self, dictionary, max_iter=False, tol=None, sparsity=None):
        self.D = np.array(dictionary.matrix)
        self.max_iter = max_iter
        self.tol = tol
        self.sparsity = sparsity
        if (self.tol is None and self.sparsity is None) or (self.tol is not None and self.sparsity is not None):
            raise ValueError("blub")
        self.data = None
        self.alphas = []

    def fit(self, Y):
        return [], self.alphas


class OrthogonalMatchingPursuit(Pursuit):
    """
    Wrapper for orthogonal_mp from scikit-learn
    """

    def fit(self, Y):
        return orthogonal_mp(self.D, Y, n_nonzero_coefs=self.sparsity,
                             tol=self.tol, precompute=True)

    
class Dictionary:
    """
    The Dictionary class is more or less a wrapper around the numpy array class. It holds a numpy ndarray in
    the attribute `matrix` and adds some useful functions for it. The dictionary elements can be accessed
    either by D.matrix[i,j] or directly through D[i,j].
    """

    def __init__(self, matrix):
        self.matrix = np.array(matrix)
        self.shape = matrix.shape

    def __getitem__(self, item):
        return self.matrix[item]

    def is_unitary(self):
        """
        Checks whether the dictionary is unitary.
        Returns:
            True, if the dicitonary is unitary.
        """
        n, K = self.shape
        if n == K:
            return np.allclose(np.dot(self.matrix.T, self.matrix), np.eye(n))
        else:
            return False

    def is_normalized(self):
        """
        Checks wheter the dictionary is l2-normalized.
        Returns:
            True, if dictionary is l2-normalized.
        """
        n, K = self.shape
        return np.allclose([np.linalg.norm(self.matrix[:, i]) for i in range(K)], np.ones(K))


    def mutual_coherence(self):
        """
        Computes the dictionary's mutual coherence.
        Returns:
            Mutual coherence
        """
        return np.max(self._mutual_coherence(self.matrix))

    @staticmethod
    def _mutual_coherence(D):
        n, K = D.shape
        mu = [np.abs(np.dot(D[:, i].T, D[:, j]) /
                     (np.linalg.norm(D[:, i]) * np.linalg.norm(D[:, j])))
              for i in range(K) for j in range(K) if j != i]
        return mu

    def to_img(self):
        """
        Transforms the dictionary columns into patches and orders them for plotting purposes.
        Returns:
            Reordered dictionary matrix
        """
        # dictionary dimensions
        D = self.matrix
        n, K = D.shape
        M = self.matrix
        # stretch atoms
        for k in range(K):
            M[:, k] = M[:, k] - (M[:, k].min())
            if M[:, k].max():
                M[:, k] = M[:, k] / D[:, k].max()

        # patch size
        n_r = int(np.sqrt(n))

        # patches per row / column
        K_r = int(np.sqrt(K))

        # we need n_r*K_r+K_r+1 pixels in each direction
        dim = n_r * K_r + K_r + 1
        V = np.ones((dim, dim)) * np.min(D)

        # compute the patches
        patches = [np.reshape(D[:, i], (n_r, n_r)) for i in range(K)]

        # place patches
        for i in range(K_r):
            for j in range(K_r):
                V[j * n_r + 1 + j:(j + 1) * n_r + 1 + j, i * n_r + 1 + i:(i + 1) * n_r + 1 + i] = patches[
                    i * K_r + j]
        return V
    
class KSVD:
    """
    Implements the original K-SVD Algorithm as described in [1].
    [1] Aharon, M., Elad, M. and Bruckstein, A., 2006. K-SVD: An algorithm for designing overcomplete dictionaries for
        sparse representation. IEEE Transactions on signal processing, 54(11), p.4311.
    Args:
        dictionary: Initial dictionary of type sparselandtools.dictionaries.Dictionary
        pursuit: Pursuit method to be used (any method from sparselandtools.pursuits)
        sparsity: Target sparsity
        noise_gain: Target noise_gain. If set, this will override the target sparsity
        sigma: Signal or image noise standard deviation.
    """

    def __init__(self, dictionary: Dictionary, pursuit: Type[Pursuit], sparsity: int, noise_gain=None, sigma=None):
        self.dictionary = Dictionary(dictionary.matrix)
        self.alphas = None
        self.pursuit = pursuit
        self.sparsity = sparsity
        self.noise_gain = noise_gain
        self.sigma = sigma
        self.original_image = None
        self.sparsity_values = []
        self.mses = []
        self.ssims = []
        self.psnrs = []
        self.iter = None

    def sparse_coding(self, Y: np.ndarray):
        logging.info("Entering sparse coding stage...")
        if self.noise_gain and self.sigma:
            p = self.pursuit(self.dictionary, tol=(self.noise_gain * self.sigma))
        else:
            p = self.pursuit(self.dictionary, sparsity=self.sparsity)
        self.alphas = p.fit(Y)
        logging.info("Sparse coding stage ended.")

    def dictionary_update(self, Y: np.ndarray):
        # iterate rows
        D = self.dictionary.matrix
        n, K = D.shape
        R = Y - D.dot(self.alphas)
        for k in range(K):
            logging.info("Updating column %s" % k)
            wk = np.nonzero(self.alphas[k, :])[0]
            if len(wk) == 0:
                continue
            Ri = R[:,wk] + D[:,k,None].dot(self.alphas[None,k,wk])
            U, s, Vh = np.linalg.svd(Ri)
            D[:, k] = U[:, 0]
            self.alphas[k, wk] = s[0] * Vh[0, :]
            R[:, wk] = Ri - D[:,k,None].dot(self.alphas[None,k,wk])
        self.dictionary = Dictionary(D)

    def fit(self, Y: np.ndarray, iter: int):
        for i in range(iter):
            logging.info("Start iteration %s" % (i + 1))
            self.sparse_coding(Y)
            self.dictionary_update(Y)
        return self.dictionary, self.alphas

def preprocess(image):
    core, factors = partial_tucker(tl.tensor(image), rank=image.shape, modes=[0, 1, 2])
    return factors[0]


def get_matrix(y, image_path, patch_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    component = preprocess(image)
    
    # Extract all reference patches from the image
    patches = extract_patches_2d(component, (patch_size, patch_size))
    data = patches.reshape(patches.shape[0], -1)
    y = np.vstack([y, data])
    
    return y


# вычисление фич
def get_features(x):
    f_mic = []
    f_mac = []
    for i in range(x.shape[0]):
        values = x[i]
        values = np.abs(values[values!=0])
        sigma, _, mean = sp.stats.lognorm.fit(values, loc=0)
        f_mic.append(np.exp(mean + 0.5*sigma**2))
        f_mac.append(values.shape[0])
    return f_mic + f_mac

In [102]:
def f_get(ni):
    v = np.load(open('dictionary1.npy', 'rb'))
    # создание таблицы для записи фич
    df = pd.read_csv('images/response.csv')
    images = df['image'].tolist()
    labels = df['label'].tolist()
    im_str='images_perm/images_'+str(ni)+'/'
    image_paths = [im_str+image for image in images]
    del df
    image_paths = image_paths[:500]
    columns = ['image']
    for i in range(v.shape[1]):
        columns.append('f%d_mic'%(i+1))
    for i in range(v.shape[1]):
        columns.append('f%d_mac'%(i+1))
    columns.append('label')
    data = {column: [] for column in columns}
    df = pd.DataFrame(data)
    patch_size=4
#     for i, image_path in enumerate(image_paths):
#         y = np.zeros(patch_size * patch_size)
#         y = get_matrix(y, image_path, patch_size)
#         y = y.T
# #     print(i)
#         X = orthogonal_mp(v, y, n_nonzero_coefs=8)
# #     print(i)
#         features = get_features(X)
#         data = [images[i]] + features + [labels[i]]
#         data = {column: [datum] for column, datum in zip(columns, data)}
#         tmp = pd.DataFrame(data)
#         df = pd.concat([df, tmp], axis=0)
        # перебираем все файлы
    for i, image_path in enumerate(image_paths):

        try:
            # создаем массив
            y = np.zeros(patch_size*patch_size)

            # получаем матрицу "y"
            y = get_matrix(y, image_path, patch_size)

            # транспонируем матрицу
            y = y.T

            # преобразование KSVD
            X = orthogonal_mp(v, y, n_nonzero_coefs=8)

            # определяем параметры изображения
            features = get_features(X)

            # список со значениями одной строки таблицы
            data = [images[i]] + features + [labels[i]]

            # записываем данные во временную таблицу
            data = {column: [datum] for column, datum in zip(columns, data)}

            tmp = pd.DataFrame(data)

            # добавляем временную таблицу в конец общей таблицы
            df = pd.concat([df, tmp], axis=0)

            print('%d/%d %s' % (i, len(image_paths), image_path))
    
        except:
            continue
    columns = [column for column in df.columns if column not in ['image', 'label']]
    x = df[columns].to_numpy()
    y = df['label'].to_numpy()
        
    try:
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8)
    except:
        return 0, 0
    return x_train, y_train

In [100]:
def permutation_test(model, ind: int, x_df: DataFrame, y_df: DataFrame, save: bool = False) -> dict:
    # finding response dependence each predictor of input data
#     np.random.seed(1024)
    df = pd.read_csv('dataset2.csv')
    columns = [column for column in df.columns if column not in ['image', 'label']]
    x = df[columns].to_numpy()
    y = df['label'].to_numpy()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8)
#     model = make_pipeline(StandardScaler(), SVR(C=35, epsilon=0.1))
#     fitness_function = model.fit(x_train, y_train.flatten())
#     fitness_function = SymbolicRegressionFitness(X_train=x_df.to_numpy(), y_train=y_df.to_numpy().flatten())
    benchmark_error = model.score(x_train, y_train)
    column_error_change = dict()
    PERMUTATIONS_NUM=1
    if save:
        name_dir = "model_{0}".format(ind)
#         model_path = PERMUTATIONS_DIR + "/" + name_dir
#         if not os.path.exists(model_path):
#             os.mkdir(PERMUTATIONS_DIR + "/" + name_dir)
    for i in range(PERMUTATIONS_NUM):
        x_df_copy = deepcopy(x_df)
        for column in x_df.columns:
            x_df_copy[column] = np.random.permutation(x_df_copy[column].values)
            if save:
                # saving dataset with EACH PERMUTATED COLUMN
                file_name = "perm_{0}_model_{1}_in_top.csv".format(i, ind)
#                 x_df_copy.to_csv(model_path + "/" + file_name, index=False, sep=";")
        for column in x_df.columns:
            print(column)
            column_res = 0 if column not in column_error_change.keys() else column_error_change[column]
            column_tmp = deepcopy(x_df[column])
            x_df[column] = x_df_copy[column]
            
#             ////////////////////
            images = transform(x_df, i)
            print(1)
            x_train, y_train = f_get(i)
            if x_train==0 and y_train==0:
                error =0
#             fitness_function.X_train = x_df.to_numpy()
            else:

                error = model.score(x_train, y_train)
            print(2)
            if save:
                error_file = "perm_col_{0}_error_{1}.txt".format(column, i)
                np.save(model_path + "/" + error_file, error)
            column_res += np.fabs(error - benchmark_error)
            x_df[column] = column_tmp
            if column_res > 1e-7:
                column_error_change.update({column: column_res})
    for column in x_df.columns:
        if column in column_error_change.keys():
            val = column_error_change[column]
            column_error_change.update({column: [val / PERMUTATIONS_NUM]})
    if save:  # save error after current permutations
        mean_error_file = "mean_error_model_{0}.txt".format(ind)  # saving error per column to csv for correct reading from R script
#         error_df = DataFrame.from_dict(column_error_change)
#         error_df.to_csv(mean_error_file, index=False, sep=";")
    return column_error_change

In [44]:
from joblib import dump, load
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
model = load('my_model.joblib')
# model = make_pipeline(StandardScaler(), model)

In [16]:
#создание датасета необходимого нам формата
import math
import pandas as pd
import numpy as np
import csv
from PIL import Image


columns = ['doy', 'geo_id', 'gr_covar1', 'gr_covar2', 'gr_covar3', 'gr_covar4', 'gr_covar5', 'gr_covar6', 'gr_covar7', 'gr_covar8', 'gr_covar9', 'gr_covar10', 'gr_covar11', 'gr_covar12', 'gr_covar13', 'gr_covar14', 'gr_covar15', 'gr_covar16', 'gr_covar17', 'gr_covar18', 'response', 'species', 'year']

w_columns = []

for i in range(25):
    w_columns.append('dl%d'%(i+1))
    w_columns.append('rain%d'%(i+1))
    w_columns.append('srad%d'%(i+1))
    w_columns.append('tmax%d'%(i+1))
    w_columns.append('tmin%d'%(i+1))
    
for w in w_columns:
    columns.append(w)

data = {column: [] for column in columns}
df = pd.DataFrame(data)

def get_weather_information(filename):
    df = pd.read_csv(filename)
    weather_info = []
    weather_groups = df.columns.values.tolist()
    # ['month', 'doy', 'geo_id', 'dl', 'rain', 'srad', 'tmax', 'tmin', 'year']
    for group in weather_groups:
        tmp = []
        for dset in df[group]:
            tmp.append(dset)
        weather_info.append(np.array(tmp))
    weather_info = np.array(weather_info).transpose()
    return weather_info


def get_chickpea_information(filename):
    df = pd.read_csv(filename)
    plant_groups = df.columns.values.tolist()
    chickpea_info = []
    # ['doy', 'geo_id', 'gr_covar', 'response', 'species', 'year']
    for group in plant_groups:
        tmp = []
        for dset in df[group]:
            if group == 'gr_covar1':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar4':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar7':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar10':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar13':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar16':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar2':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar5':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar8':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar11':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar14':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar17':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar3':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar6':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar9':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar12':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar15':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar18':
                if (dset == 1.):
                    dset = 1
            tmp.append(dset)
        chickpea_info.append(np.array(tmp))
    chickpea_info = list(map(list, zip(*chickpea_info)))
    plants = []
    for chickpea in chickpea_info:
        plants.append(chickpea)
    return plants


In [ ]:
def get_data(filename_chickpea, filename_weather, df, columns): 
    tmp_weather_first = get_weather_information(filename_weather)
    weather = []
    for i in range(len(tmp_weather_first)):
        weather.append(tmp_weather_first[i])
    all_chickpea = get_chickpea_information(filename_chickpea)
    for plant in all_chickpea:
        data = plant
        days_data=[]
        
        for day in weather:
            if (day[-1] == plant[-1]) and (day[2] == plant[1]) and (plant[0] - 5 <= day[1] < plant[0] + 20):
                for i in range(3, 8):
                    days_data.append(day[i])
                    
        data = data + days_data
        data = {column: [datum] for column, datum in zip(columns, data)}
        tmp = pd.DataFrame(data)
        df = pd.concat([df, tmp], axis=0)
        
    return df        
                    

chickpea = "chickpea-ssm-snp.csv"
weather = "weather.csv"
df = get_data(chickpea, weather, df, columns)

In [17]:
df

,doy,geo_id,gr_covar1,gr_covar2,gr_covar3,gr_covar4,gr_covar5,gr_covar6,gr_covar7,gr_covar8,...,dl24,rain24,srad24,tmax24,tmin24,dl25,rain25,srad25,tmax25,tmin25
0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,161.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,161.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,32.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,32.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,32.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,32.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9


In [18]:
x_df = df.loc[:, df.columns != 'doy']

In [19]:
x_df

,geo_id,gr_covar1,gr_covar2,gr_covar3,gr_covar4,gr_covar5,gr_covar6,gr_covar7,gr_covar8,gr_covar9,...,dl24,rain24,srad24,tmax24,tmin24,dl25,rain25,srad25,tmax25,tmin25
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,...,10.068720,0.0,7.51,17.7,7.0,10.073209,17.0,8.38,17.2,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.952243,0.0,7.13,22.0,11.3,10.989847,0.0,6.87,21.6,10.9


In [20]:
y_df = df.loc[:, df.columns == 'doy']

In [21]:
y_df

,doy
0,161.0
0,161.0
0,161.0
0,161.0
0,161.0
...,...
0,32.0
0,32.0
0,32.0
0,32.0


In [103]:
save = False
perm_res = permutation_test(model, 1, df, y_df, save=save)

doy
1
2
geo_id
1
2
gr_covar1
1
2
gr_covar2
1
2
gr_covar3
1
2
gr_covar4
1
2
gr_covar5
1
2
gr_covar6
1
2
gr_covar7
1
2
gr_covar8
1
2
gr_covar9
1
2
gr_covar10
1
2
gr_covar11
1
2
gr_covar12
1
2
gr_covar13
1
2
gr_covar14
1
2
gr_covar15
1
2
gr_covar16
1
2
gr_covar17
1
2
gr_covar18
1
2
response
1
2
species
1
2
year
1
2
dl1
1
2
rain1
1
2
srad1
1
2
tmax1
1
2
tmin1
1
2
dl2
1
2
rain2
1
0/500 images_perm/images_0/image1.png
1/500 images_perm/images_0/image2.png
2/500 images_perm/images_0/image3.png
3/500 images_perm/images_0/image4.png
4/500 images_perm/images_0/image5.png
5/500 images_perm/images_0/image6.png
6/500 images_perm/images_0/image7.png
7/500 images_perm/images_0/image8.png
8/500 images_perm/images_0/image9.png
9/500 images_perm/images_0/image10.png
10/500 images_perm/images_0/image11.png
11/500 images_perm/images_0/image12.png
12/500 images_perm/images_0/image13.png
13/500 images_perm/images_0/image14.png
14/500 images_perm/images_0/image15.png
15/500 images_perm/images_0/image16.png
1

192/500 images_perm/images_0/image193.png
193/500 images_perm/images_0/image194.png
194/500 images_perm/images_0/image195.png
195/500 images_perm/images_0/image196.png
196/500 images_perm/images_0/image197.png
197/500 images_perm/images_0/image198.png
198/500 images_perm/images_0/image199.png
199/500 images_perm/images_0/image200.png
200/500 images_perm/images_0/image201.png
201/500 images_perm/images_0/image202.png
202/500 images_perm/images_0/image203.png
203/500 images_perm/images_0/image204.png
204/500 images_perm/images_0/image205.png
205/500 images_perm/images_0/image206.png
206/500 images_perm/images_0/image207.png
207/500 images_perm/images_0/image208.png
208/500 images_perm/images_0/image209.png
209/500 images_perm/images_0/image210.png
210/500 images_perm/images_0/image211.png
211/500 images_perm/images_0/image212.png
212/500 images_perm/images_0/image213.png
213/500 images_perm/images_0/image214.png
214/500 images_perm/images_0/image215.png
215/500 images_perm/images_0/image

388/500 images_perm/images_0/image389.png
389/500 images_perm/images_0/image390.png
390/500 images_perm/images_0/image391.png
391/500 images_perm/images_0/image392.png
392/500 images_perm/images_0/image393.png
393/500 images_perm/images_0/image394.png
394/500 images_perm/images_0/image395.png
395/500 images_perm/images_0/image396.png
396/500 images_perm/images_0/image397.png
397/500 images_perm/images_0/image398.png
398/500 images_perm/images_0/image399.png
399/500 images_perm/images_0/image400.png
400/500 images_perm/images_0/image401.png
401/500 images_perm/images_0/image402.png
402/500 images_perm/images_0/image403.png
403/500 images_perm/images_0/image404.png
404/500 images_perm/images_0/image405.png
405/500 images_perm/images_0/image406.png
406/500 images_perm/images_0/image407.png
407/500 images_perm/images_0/image408.png
408/500 images_perm/images_0/image409.png
409/500 images_perm/images_0/image410.png
410/500 images_perm/images_0/image411.png
411/500 images_perm/images_0/image

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [104]:
perm_res

NameError: name 'perm_res' is not defined